In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from stargazer.stargazer import Stargazer

# Summary Statistics

In [56]:
Summary_Stats

,National,GDP,Approvals,SandP,MVR,CPI,UP
count,25.000,25.000,25.000,25.000,25.000,25.000,25.000
mean,0.051,0.005,-0.020,0.069,-0.003,0.001,-0.002
std,0.061,0.004,0.143,0.157,0.010,0.009,0.006
min,-0.067,-0.002,-0.287,-0.413,-0.021,-0.018,-0.019
25%,0.021,0.001,-0.135,0.011,-0.009,-0.005,-0.006
50%,0.048,0.004,-0.007,0.084,-0.002,0.002,-0.004
75%,0.086,0.009,0.083,0.151,0.005,0.005,0.002
max,0.166,0.012,0.254,0.351,0.013,0.024,0.009


# Correlation Matrix

In [57]:
Correlation_Matrix

,National,GDP,Approvals,SandP,MVR,CPI,UP
National,1.00,-0.10,-0.19,-0.13,-0.06,0.27,0.05
GDP,-0.10,1.00,0.30,0.39,0.09,-0.14,-0.49
Approvals,-0.19,0.30,1.00,0.42,-0.13,-0.17,-0.11
SandP,-0.13,0.39,0.42,1.00,-0.18,-0.22,-0.44
MVR,-0.06,0.09,-0.13,-0.18,1.00,0.52,0.52
CPI,0.27,-0.14,-0.17,-0.22,0.52,1.00,0.55
UP,0.05,-0.49,-0.11,-0.44,0.52,0.55,1.00


# Results

In [58]:
stargazer

In [4]:
Start = "1994"
End = "2019"`

In [5]:
gccsas = ['Rest of NSW', 'Greater Sydney', 'Rest of Vic.',
       'Greater Melbourne', 'Greater Brisbane', 'Rest of Qld',
       'Greater Adelaide', 'Rest of SA', 'Rest of WA', 'Greater Perth',
       'Greater Hobart', 'Rest of Tas.', 'Greater Darwin', 'Rest of NT',
       'Australian Capital Territory']
#gccsas = ['Greater Sydney','Greater Melbourne', 'Greater Brisbane','Greater Adelaide', 'Greater Perth',
#       'Greater Hobart', 'Greater Darwin','Australian Capital Territory']

In [31]:
def Date_Stripper(df):
    df = df.reset_index()
    df['Date'] = df['Date'].apply(lambda df: df.strftime('%Y-%m'))
    df.set_index('Date', inplace=True)
    return(df)

def SA3_Data_Extractor(GCCSA_Name, Housing_Type, Start, End):
    #Function returns log returns
    df = pd.read_csv("./Data/Indexes/SA3.csv")
    Dates = pd.to_datetime(df['Row Labels'][3:])
    df = df[df.columns[df.iloc[1] == GCCSA_Name]]
    df = df[df.columns[df.iloc[2] == Housing_Type]]
    df.columns = df.iloc[0]
    df = df[3:]
    df['Date'] = Dates
    df = df[df.Date.dt.month.isin(['12'])]
    df.index = df.Date
    df = df.rename_axis(None, axis=1).rename_axis('Date', axis=0)
    df = df.apply(pd.to_numeric)
    df = np.log(1 + df.pct_change()[1:])
    df = df.dropna(axis=1,how='all')
    del df['Date']
    df = Date_Stripper(df)
    df = df[Start:End]
    return(df)

def SA3_Level_Returns(Start, End, Type, gccsas):
    count = 0
    for gccsa in gccsas:
        if count == 0:
            df = SA3_Data_Extractor(gccsa, Type, Start, End)
            count = count + 1
        else:
            temp = SA3_Data_Extractor(gccsa, Type, Start, End)
            df = pd.concat([df, temp], axis=1)
    return(df)

def National_Data_Extractor(Start, End, Type):
    df = pd.read_csv("./Data/Indexes/National.csv")
    df.Date = pd.to_datetime(df.Date)
    df = df[df.Date.dt.month.isin(['12'])]
    df.index = df.Date
    del df['Date']
    df = df.apply(pd.to_numeric)
    df = np.log(1 + df.pct_change()[1:])
    df = Date_Stripper(df)
    df = df[Start:End]
    df['National'] = df[Type]
    return(df)

def GDP_Macro_Variables_Exctractor(Start, End):
    df = pd.read_csv("./Data/Macro_Variables/GDP_Per_Capita_Trend.csv")
    df.columns = ['Date', 'GDP']
    df.Date = pd.to_datetime(df['Date'])
    df = df[df.Date.dt.month.isin(['12'])]
    df.index = df.Date
    del df['Date']
    df['GDP t-1'] = df["GDP"].shift(1)
    df = df.dropna() 
    df = Date_Stripper(df)
    df = df[Start:End]
    #df = np.log(1 + df/100)
    df = df/100
    return(df['GDP t-1'])

def Approvals_Extractor(Start, End):
    #1990 only has a December entry
    df = pd.read_csv("./Data/Macro_Variables/Building_Aprovals.csv")
    df = df.iloc[::-1]
    df.Date = pd.to_datetime(df['Date'])
    df.Approvals = 1 + df.Approvals/100
    g = df.groupby(df.Date.dt.year)
    df = g.aggregate({"Approvals":np.prod})
    df['Date'] = pd.to_datetime(df.index, format='%Y')
    df.index = df.Date
    del df['Date']
    df = Date_Stripper(df)
    df.columns = ['Approvals']
    df['Approvals t-1'] = df["Approvals"].shift(1)
    df = df[Start:End]
    df = df - 1
    return(df['Approvals t-1'])

def SP200_Exctractor(Start, End):
    df = pd.read_csv("./Data/Macro_Variables/ASX200.csv")
    df = df.iloc[::-1]
    df.columns = ['Date', 'ASX']
    df = df.dropna()
    df.Date = pd.to_datetime(df.Date)
    df = df[df.Date.dt.month.isin(['12'])]
    df.index = pd.to_datetime(df['Date'])
    del df['Date']
    df = df.pct_change()
    df['ASX t-1'] = df["ASX"].shift(1)
    #df = np.log(1 + df)
    df = Date_Stripper(df)
    df = df[Start:End]
    return(df['ASX t-1'])

def MVR_Macro_Variables_Exctractor(Start, End):
    df = pd.read_csv("./Data/Macro_Variables/Mortgage_Variable_Rate.csv")
    df.columns = ['Date', 'MVR']
    df.Date = pd.to_datetime(df.Date)
    df = df[df.Date.dt.month.isin(['12'])]
    df.index = pd.to_datetime(df['Date'])
    del df['Date']
    df = df.diff()[1:]
    df['MVR t-1'] = df["MVR"].shift(2)
    df = df.dropna() 
    df = Date_Stripper(df)
    df = df[Start:End]
    #df = np.log(1 + df/100)
    df = df/100
    return(df)

def Unexpected_CPI(Start, End):
    df = pd.read_csv("./Data/Macro_Variables/CPI.csv")
    df = df.iloc[::-1]
    df.Date = pd.to_datetime(df.Date)
    df = df[df.Date.dt.month.isin(['12'])]
    df.index = df.Date
    del df['Date']
    df['CPI_Lag'] = df.shift(1)
    df = df[1:]
    y = df.CPI.values
    x = df.CPI_Lag
    results = sm.OLS(y,x).fit()
    df = results.resid
    df = df.shift(1)
    df = Date_Stripper(df)
    df = df[Start:End]
    return(df/100)

def UP(Start, End):
    df = pd.read_csv("./Data/Macro_Variables/Unemployment.csv")
    df = df.iloc[::-1]
    df.Date = pd.to_datetime(df.Date)
    df = df[df.Date.dt.month.isin(['12'])]
    df.index = df.Date
    del df['Date']
    df.UP = df.diff()/100
    df = Date_Stripper(df)
    df = df[Start:End]
    return(df)

def AGB10(Start, End):
    df = pd.read_csv("./Data/Macro_Variables/10_Year.csv")
    df = df.iloc[::-1]
    df.Date = pd.to_datetime(df.Date)
    df = df[df.Date.dt.month.isin(['12'])]
    df.index = df.Date
    del df['Date']
    df.UP = df.diff()/100
    df = Date_Stripper(df)
    df = df[Start:End]
    return(df)

# Data Exploration

In [32]:
n = 25
columns = ['National', 'GDP', 'Approvals', 'SandP', 'MVR', 'CPI', 'UP']
X = pd.DataFrame(0, index=np.arange(n), columns=columns)
X.National = National_Data_Extractor(Start, End, "D")['National'].values
X.GDP = GDP_Macro_Variables_Exctractor(Start, End).values
X.Approvals = Approvals_Extractor(Start, End).values
X.SandP = SP200_Exctractor(Start, End).values
X.MVR = MVR_Macro_Variables_Exctractor(Start, End)['MVR t-1'].values
X.CPI = Unexpected_CPI(Start, End).values
X.UP = UP(Start, End).values
X = sm.add_constant(X)

## Summary Statistics

In [37]:
Summary_Stats = X.iloc[:,1:].describe().round(3)
Summary_Stats

,National,GDP,Approvals,SandP,MVR,CPI,UP
count,25.000,25.000,25.000,25.000,25.000,25.000,25.000
mean,0.051,0.005,-0.020,0.069,-0.003,0.001,-0.002
std,0.061,0.004,0.143,0.157,0.010,0.009,0.006
min,-0.067,-0.002,-0.287,-0.413,-0.021,-0.018,-0.019
25%,0.021,0.001,-0.135,0.011,-0.009,-0.005,-0.006
50%,0.048,0.004,-0.007,0.084,-0.002,0.002,-0.004
75%,0.086,0.009,0.083,0.151,0.005,0.005,0.002
max,0.166,0.012,0.254,0.351,0.013,0.024,0.009


## Correlation Matrix

In [41]:
Correlation_Matrix = X.iloc[:,1:].corr().round(2)
Correlation_Matrix

,National,GDP,Approvals,SandP,MVR,CPI,UP
National,1.00,-0.10,-0.19,-0.13,-0.06,0.27,0.05
GDP,-0.10,1.00,0.30,0.39,0.09,-0.14,-0.49
Approvals,-0.19,0.30,1.00,0.42,-0.13,-0.17,-0.11
SandP,-0.13,0.39,0.42,1.00,-0.18,-0.22,-0.44
MVR,-0.06,0.09,-0.13,-0.18,1.00,0.52,0.52
CPI,0.27,-0.14,-0.17,-0.22,0.52,1.00,0.55
UP,0.05,-0.49,-0.11,-0.44,0.52,0.55,1.00


In [43]:
Y = SA3_Level_Returns(Start, End, "D", gccsas)
suburbs = Y.columns

In [46]:
def Model1(Vars):
    risk_factors = []
    resid = []
    x = X[Vars]
    x = sm.add_constant(x)
    for suburb in suburbs:
        y = Y[suburb].values
        results = sm.OLS(y,x).fit()
        risk_factors.append(results.params[1:])
        s2 = np.mean(results.resid**2)
        residuals.append(s2)
    first_pass = pd.DataFrame(pd.DataFrame(risk_factors))
    first_pass.index = suburbs
    first_pass['Average_Returns'] = np.mean(Y).values
    factors = first_pass[Vars]
    factors = sm.add_constant(factors)                
    cross_section = sm.OLS(first_pass.Average_Returns,factors).fit()
    return(cross_section)

In [55]:
Vars = ['GDP']
model_1 = Model1(Vars)
Vars = ['Approvals']
model_2 = Model1(Vars)
Vars = ['National']
model_3 = Model1(Vars)
Vars = ['SandP']
model_4 = Model1(Vars)
Vars = ['MVR']
model_5 = Model1(Vars)
Vars = ['CPI']
model_6 = Model1(Vars)
Vars = X.iloc[:,1:].columns
model_7 = Model1(Vars)


stargazer = Stargazer([model_1, model_2, model_3, model_4, model_5, model_6, model_7])